# Comprehensive SyftBox Permissions Tutorial
## Part 5: Mastery - Complex Scenarios and Expert Techniques

Welcome to the final tutorial! You've learned all the individual pieces. Now let's put them together to solve complex real-world scenarios and become a true SyftBox permissions expert.

### What You'll Master
- Complex multi-requirement scenarios
- Advanced debugging and troubleshooting
- Performance optimization techniques
- Best practices and design patterns
- Expert-level edge cases

### Prerequisites
- Completed all previous parts (1-4)
- Strong understanding of patterns, inheritance, terminals, and limits
- Ready to think like a permissions architect!

In [ ]:
import syft_perm
from pathlib import Path
import tempfile
import shutil
import yaml
import time
from collections import defaultdict

# Setup workspace
tutorial_dir = Path(tempfile.mkdtemp(prefix="mastery_tutorial_"))
print(f"Tutorial workspace: {tutorial_dir}")

def cleanup_and_reset():
    global tutorial_dir
    if tutorial_dir.exists():
        shutil.rmtree(tutorial_dir)
    tutorial_dir = Path(tempfile.mkdtemp(prefix="mastery_tutorial_"))
    print(f"Fresh workspace: {tutorial_dir}")
    return tutorial_dir

def show_yaml(path, title=None):
    yaml_file = path / "syft.pub.yaml"
    if yaml_file.exists():
        display_title = title or f"syft.pub.yaml in {path.name}"
        print(f"\n=== {display_title} ===")
        print(yaml_file.read_text())
        print("=" * 50)

def audit_permissions(directory, users):
    """Comprehensive permission audit for a directory tree"""
    print(f"\n=== Permission Audit: {directory.relative_to(tutorial_dir)} ===")
    
    # Find all files
    files = list(directory.rglob("*"))
    files = [f for f in files if f.is_file() and f.name != "syft.pub.yaml"]
    
    # Create audit matrix
    audit_results = defaultdict(dict)
    
    for file_path in files:
        syft_file = syft_perm.open(file_path)
        rel_path = file_path.relative_to(tutorial_dir)
        
        for user in users:
            permissions = {
                'read': syft_file.has_read_access(user),
                'create': syft_file.has_create_access(user),
                'write': syft_file.has_write_access(user),
                'admin': syft_file.has_admin_access(user)
            }
            audit_results[str(rel_path)][user] = permissions
    
    return audit_results

def performance_test(file_path, operations=100):
    """Test permission check performance"""
    syft_file = syft_perm.open(file_path)
    
    start_time = time.time()
    for _ in range(operations):
        syft_file.has_read_access("test@example.com")
    end_time = time.time()
    
    total_time = end_time - start_time
    avg_time = total_time / operations * 1000  # ms per operation
    
    return {
        'total_time': total_time,
        'avg_time_ms': avg_time,
        'operations': operations
    }

## Chapter 1: Complex Multi-Tenant Scenario

**Scenario**: Build a multi-tenant SaaS platform where:
- Each tenant has isolated data
- Different user roles within tenants
- Shared resources with different access levels
- Admin oversight capabilities
- File size limits based on subscription tier

In [ ]:
# Create multi-tenant SaaS structure
# Structure:
# saas_platform/
#   ├── shared/
#   │   ├── templates/
#   │   └── documentation/
#   ├── tenants/
#   │   ├── acme_corp/ (enterprise)
#   │   │   ├── private/
#   │   │   ├── shared/
#   │   │   └── uploads/
#   │   ├── startup_inc/ (premium)
#   │   │   ├── private/
#   │   │   └── uploads/
#   │   └── freelancer/ (basic)
#   │       └── uploads/
#   └── admin/
#       ├── logs/
#       └── monitoring/

saas_structure = {
    # Shared resources
    "shared/templates/email_template.html": "Email template",
    "shared/documentation/api_docs.md": "API documentation",
    
    # Acme Corp (Enterprise)
    "tenants/acme_corp/private/customer_data.csv": "Customer database",
    "tenants/acme_corp/private/financial_reports.xlsx": "Financial reports",
    "tenants/acme_corp/shared/team_docs.md": "Team documentation",
    "tenants/acme_corp/uploads/presentation.pptx": "Large presentation file",
    
    # Startup Inc (Premium)
    "tenants/startup_inc/private/business_plan.pdf": "Business plan",
    "tenants/startup_inc/uploads/product_demo.mp4": "Product demo video",
    
    # Freelancer (Basic)
    "tenants/freelancer/uploads/portfolio.zip": "Portfolio archive",
    
    # Admin area
    "admin/logs/access.log": "Access logs",
    "admin/monitoring/metrics.json": "Platform metrics"
}

# Create all files
for rel_path, content in saas_structure.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Create files with different sizes based on type
    if "large" in content.lower() or ".pptx" in rel_path:
        full_path.write_text("x" * 200000)  # 200KB
    elif ".mp4" in rel_path or ".zip" in rel_path:
        full_path.write_text("x" * 50000)   # 50KB
    else:
        full_path.write_text("x" * 5000)    # 5KB

print(f"Created multi-tenant SaaS platform with {len(saas_structure)} files")
print("\nTenant tiers:")
print("• acme_corp: Enterprise (unlimited uploads)")
print("• startup_inc: Premium (100KB upload limit)")
print("• freelancer: Basic (10KB upload limit)")

In [ ]:
# Set up complex permission hierarchy

# Platform root: Admin oversight
platform_yaml = tutorial_dir / "syft.pub.yaml"
platform_yaml.write_text("""rules:
- pattern: "**"
  access:
    admin:
    - platform_admin@saas.com
""")

# Shared resources: Public read access
shared_yaml = tutorial_dir / "shared" / "syft.pub.yaml"
shared_yaml.write_text("""rules:
- pattern: "**"
  access:
    read:
    - "*"  # All tenants can read shared resources
""")

# Acme Corp (Enterprise): Terminal with full access
acme_yaml = tutorial_dir / "tenants" / "acme_corp" / "syft.pub.yaml"
acme_yaml.write_text("""terminal: true
rules:
- pattern: "private/**"
  access:
    admin:
    - acme_admin@acme.com
    write:
    - acme_manager@acme.com
    read:
    - acme_employee@acme.com
- pattern: "shared/**"
  access:
    write:
    - acme_employee@acme.com
- pattern: "uploads/**"
  access:
    create:
    - acme_employee@acme.com
  # No limits = unlimited for enterprise
""")

# Startup Inc (Premium): Terminal with upload limits
startup_yaml = tutorial_dir / "tenants" / "startup_inc" / "syft.pub.yaml"
startup_yaml.write_text("""terminal: true
rules:
- pattern: "private/**"
  access:
    admin:
    - startup_owner@startup.com
    read:
    - startup_team@startup.com
- pattern: "uploads/**"
  access:
    create:
    - startup_team@startup.com
  limits:
    max_file_size: 102400  # 100KB limit
    allow_dirs: false
""")

# Freelancer (Basic): Terminal with strict limits
freelancer_yaml = tutorial_dir / "tenants" / "freelancer" / "syft.pub.yaml"
freelancer_yaml.write_text("""terminal: true
rules:
- pattern: "uploads/**"
  access:
    create:
    - freelancer@email.com
  limits:
    max_file_size: 10240   # 10KB limit
    allow_dirs: false
    allow_symlinks: false
""")

# Admin area: Terminal for platform admins only
admin_yaml = tutorial_dir / "admin" / "syft.pub.yaml"
admin_yaml.write_text("""terminal: true
rules:
- pattern: "**"
  access:
    admin:
    - platform_admin@saas.com
    read:
    - platform_support@saas.com
""")

print("Set up multi-tenant permission hierarchy:")
show_yaml(tutorial_dir, "Platform: Admin oversight")
show_yaml(tutorial_dir / "shared", "Shared: Public read")
show_yaml(tutorial_dir / "tenants" / "acme_corp", "Acme Corp: Enterprise (no limits)")
show_yaml(tutorial_dir / "tenants" / "startup_inc", "Startup Inc: Premium (100KB limit)")
show_yaml(tutorial_dir / "tenants" / "freelancer", "Freelancer: Basic (10KB limit)")
show_yaml(tutorial_dir / "admin", "Admin: Platform only")

In [ ]:
# Test the multi-tenant access control
print("\n=== Multi-Tenant Access Control Test ===")

# Define test users
test_users = [
    "platform_admin@saas.com",
    "acme_admin@acme.com",
    "acme_employee@acme.com",
    "startup_owner@startup.com",
    "startup_team@startup.com",
    "freelancer@email.com",
    "random_user@internet.com"
]

# Critical test cases
test_cases = [
    # (file, user, permission, expected, reason)
    ("shared/templates/email_template.html", "acme_employee@acme.com", "read", True, "Shared resources are public"),
    ("shared/templates/email_template.html", "random_user@internet.com", "read", True, "Shared resources are public"),
    
    ("tenants/acme_corp/private/customer_data.csv", "acme_admin@acme.com", "admin", True, "Acme admin has full access"),
    ("tenants/acme_corp/private/customer_data.csv", "startup_owner@startup.com", "read", False, "Cross-tenant isolation"),
    ("tenants/acme_corp/private/customer_data.csv", "platform_admin@saas.com", "admin", False, "Terminal blocks platform admin"),
    
    ("tenants/acme_corp/uploads/presentation.pptx", "acme_employee@acme.com", "create", True, "Enterprise unlimited uploads"),
    ("tenants/startup_inc/uploads/product_demo.mp4", "startup_team@startup.com", "create", False, "Premium upload too large"),
    ("tenants/freelancer/uploads/portfolio.zip", "freelancer@email.com", "create", False, "Basic upload too large"),
    
    ("admin/logs/access.log", "platform_admin@saas.com", "admin", True, "Platform admin has admin access"),
    ("admin/logs/access.log", "acme_admin@acme.com", "read", False, "Tenant admin blocked from platform admin")
]

passed = 0
total = len(test_cases)

for rel_path, user, permission, expected, reason in test_cases:
    full_path = tutorial_dir / rel_path
    syft_file = syft_perm.open(full_path)
    
    has_perm = getattr(syft_file, f"has_{permission}_access")(user)
    
    status = "✓" if has_perm == expected else "✗ FAILED"
    if has_perm == expected:
        passed += 1
    
    user_short = user.split('@')[0]
    print(f"{status} {rel_path}")
    print(f"    {user_short} → {permission}: {has_perm} ({reason})")
    
    if has_perm != expected:
        has_access, reasons = syft_file._check_permission_with_reasons(user, permission)
        print(f"    DEBUG: {reasons[0] if reasons else 'No reasons'}")
    print()

print(f"\n📊 Test Results: {passed}/{total} passed ({passed/total*100:.1f}%)")
if passed == total:
    print("🎉 Perfect! Multi-tenant isolation working correctly.")
else:
    print("❌ Some tests failed - review configuration.")

## Chapter 2: Advanced Debugging Techniques

When permissions get complex, debugging becomes critical. Let's explore advanced techniques.

In [ ]:
def advanced_permission_debugger(file_path, user, permission="read"):
    """Advanced debugging tool for complex permission issues"""
    print(f"\n🔍 ADVANCED PERMISSION DEBUGGER")
    print(f"File: {file_path.relative_to(tutorial_dir)}")
    print(f"User: {user}")
    print(f"Permission: {permission}")
    print("=" * 60)
    
    syft_file = syft_perm.open(file_path)
    
    # Step 1: Basic access check
    has_access = getattr(syft_file, f"has_{permission}_access")(user)
    print(f"\n1. 📋 RESULT: {has_access}")
    
    # Step 2: Detailed reasoning
    has_perm, reasons = syft_file._check_permission_with_reasons(user, permission)
    print(f"\n2. 🧠 REASONING:")
    for i, reason in enumerate(reasons, 1):
        print(f"   {i}. {reason}")
    
    # Step 3: Permission hierarchy analysis
    print(f"\n3. 🏗️  PERMISSION HIERARCHY:")
    all_perms = syft_file._get_all_permissions()
    for perm_level in ["admin", "write", "create", "read"]:
        users_with_perm = all_perms.get(perm_level, [])
        has_this_perm = getattr(syft_file, f"has_{perm_level}_access")(user)
        status = "✓" if has_this_perm else "✗"
        print(f"   {status} {perm_level.upper()}: {users_with_perm}")
    
    # Step 4: Inheritance path analysis
    print(f"\n4. 🌳 INHERITANCE PATH:")
    current_dir = file_path.parent
    yaml_configs = []
    
    while current_dir != tutorial_dir.parent:
        yaml_file = current_dir / "syft.pub.yaml"
        if yaml_file.exists():
            rel_path = current_dir.relative_to(tutorial_dir) if current_dir != tutorial_dir else "(root)"
            
            # Parse YAML to check for terminal and matching patterns
            try:
                with open(yaml_file, 'r') as f:
                    config = yaml.safe_load(f) or {}
                
                is_terminal = config.get("terminal", False)
                rules = config.get("rules", [])
                
                # Check if any patterns match our file
                file_rel_to_yaml = file_path.relative_to(current_dir)
                matching_patterns = []
                for rule in rules:
                    pattern = rule.get("pattern", "")
                    # Simple pattern matching check (not perfect, but good for debugging)
                    if pattern == "**" or pattern in str(file_rel_to_yaml) or str(file_rel_to_yaml).endswith(pattern.replace("*", "")):
                        matching_patterns.append(pattern)
                
                terminal_marker = " [TERMINAL]" if is_terminal else ""
                pattern_info = f" Patterns: {matching_patterns}" if matching_patterns else " No matching patterns"
                
                print(f"   📁 {rel_path}{terminal_marker}{pattern_info}")
                yaml_configs.append((current_dir, config, matching_patterns))
                
                # Stop at terminal
                if is_terminal:
                    print(f"      🛑 Inheritance stops here (terminal node)")
                    break
                    
            except Exception as e:
                print(f"   ❌ Error reading {yaml_file}: {e}")
        
        current_dir = current_dir.parent
    
    if not yaml_configs:
        print(f"   📭 No YAML configurations found in inheritance path")
    
    # Step 5: File size and limits check
    if file_path.exists():
        file_size = file_path.stat().st_size
        limits = syft_file.get_file_limits()
        
        print(f"\n5. 📏 FILE SIZE & LIMITS:")
        print(f"   File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")
        
        if limits["has_limits"]:
            max_size = limits["max_file_size"]
            if max_size:
                exceeds_limit = file_size > max_size
                status = "❌ EXCEEDS" if exceeds_limit else "✅ OK"
                print(f"   Max allowed: {max_size:,} bytes ({max_size/1024:.1f} KB) - {status}")
            else:
                print(f"   Max allowed: Unlimited")
            
            print(f"   Allow directories: {limits['allow_dirs']}")
            print(f"   Allow symlinks: {limits['allow_symlinks']}")
        else:
            print(f"   No limits configured")
    
    # Step 6: Suggestions
    print(f"\n6. 💡 DEBUGGING SUGGESTIONS:")
    if not has_access:
        if "No permission found" in str(reasons):
            print(f"   • No matching permission rules found")
            print(f"   • Check pattern syntax in YAML files")
            print(f"   • Verify user is spelled correctly")
        elif "terminal" in str(reasons).lower():
            print(f"   • Blocked by terminal node")
            print(f"   • Add permissions within the terminal directory")
        elif "limit" in str(reasons).lower():
            print(f"   • File exceeds size or type limits")
            print(f"   • Increase limits or reduce file size")
        else:
            print(f"   • Check permission hierarchy (user might have lower level)")
            print(f"   • Verify inheritance path is correct")
    else:
        print(f"   • ✅ Access working correctly")
        print(f"   • Permission granted via: {reasons[0] if reasons else 'Unknown'}")
    
    return {
        'has_access': has_access,
        'reasons': reasons,
        'permissions': all_perms,
        'yaml_configs': yaml_configs,
        'limits': syft_file.get_file_limits()
    }

# Test the advanced debugger
debug_file = tutorial_dir / "tenants" / "startup_inc" / "uploads" / "product_demo.mp4"
debug_result = advanced_permission_debugger(debug_file, "startup_team@startup.com", "create")

## Chapter 3: Performance Optimization

As permission systems grow complex, performance becomes important. Let's explore optimization techniques.

In [ ]:
# Create a performance test scenario
cleanup_and_reset()

# Create a large directory structure for performance testing
print("Creating large directory structure for performance testing...")

# Structure with many files at different depths
perf_files = []
for dept in ["engineering", "marketing", "sales", "support"]:
    dept_dir = tutorial_dir / dept
    dept_dir.mkdir()
    
    # Create YAML at department level
    dept_yaml = dept_dir / "syft.pub.yaml"
    dept_yaml.write_text(f"""rules:
- pattern: "**"
  access:
    read:
    - {dept}@company.com
""")
    
    # Create nested files
    for team in ["team_a", "team_b", "team_c"]:
        team_dir = dept_dir / team
        team_dir.mkdir()
        
        for project in ["project_1", "project_2"]:
            project_dir = team_dir / project
            project_dir.mkdir()
            
            for i in range(5):  # 5 files per project
                file_path = project_dir / f"file_{i}.txt"
                file_path.write_text(f"Content for {file_path}")
                perf_files.append(file_path)

print(f"Created {len(perf_files)} files for performance testing")
print(f"Directory depth: 4 levels")
print(f"YAML files: 4 (one per department)")

In [ ]:
# Test performance with and without caching
print("\n=== Performance Testing ===")

# Test 1: Cold start (no cache)
syft_perm.clear_permission_cache()
test_file = perf_files[0]
cold_start = performance_test(test_file, 10)

print(f"\n1. Cold Start Performance (no cache):")
print(f"   Operations: {cold_start['operations']}")
print(f"   Total time: {cold_start['total_time']:.3f}s")
print(f"   Average per operation: {cold_start['avg_time_ms']:.2f}ms")

# Test 2: Warm cache (same file)
warm_cache = performance_test(test_file, 100)

print(f"\n2. Warm Cache Performance (same file):")
print(f"   Operations: {warm_cache['operations']}")
print(f"   Total time: {warm_cache['total_time']:.3f}s")
print(f"   Average per operation: {warm_cache['avg_time_ms']:.2f}ms")
print(f"   Speedup: {cold_start['avg_time_ms'] / warm_cache['avg_time_ms']:.1f}x faster")

# Test 3: Different files (cache efficiency)
start_time = time.time()
for i in range(min(50, len(perf_files))):
    syft_file = syft_perm.open(perf_files[i])
    syft_file.has_read_access("engineering@company.com")
end_time = time.time()

multi_file_time = end_time - start_time
avg_multi_file = multi_file_time / 50 * 1000

print(f"\n3. Multiple Files Performance (cache reuse):")
print(f"   Files tested: 50")
print(f"   Total time: {multi_file_time:.3f}s")
print(f"   Average per file: {avg_multi_file:.2f}ms")

# Cache statistics
cache_stats = syft_perm.get_cache_stats()
print(f"\n4. Cache Statistics:")
print(f"   Cache size: {cache_stats['size']} entries")
print(f"   Max size: {cache_stats['max_size']} entries")
print(f"   Efficiency: {cache_stats['size'] / min(cache_stats['max_size'], 50) * 100:.1f}%")

In [ ]:
# Performance optimization recommendations
print("\n=== Performance Optimization Recommendations ===")
print()

recommendations = [
    {
        "category": "YAML Structure",
        "tips": [
            "Place YAML files as close to files as possible",
            "Use specific patterns instead of catch-all '**'",
            "Minimize the number of rules per YAML file",
            "Avoid deep directory nesting when possible"
        ]
    },
    {
        "category": "Cache Optimization", 
        "tips": [
            "Group related files in same directories",
            "Use terminal nodes to limit inheritance traversal",
            "Batch permission checks when possible",
            "Clear cache only when permissions change"
        ]
    },
    {
        "category": "Pattern Design",
        "tips": [
            "Use exact patterns for frequently accessed files",
            "Avoid complex regex-like patterns",
            "Order rules by specificity (most specific first)",
            "Use '**' patterns sparingly"
        ]
    },
    {
        "category": "Architecture",
        "tips": [
            "Design inheritance hierarchy carefully",
            "Use terminal nodes for security boundaries",
            "Limit directory depth to 5-6 levels max",
            "Consider file organization for permission efficiency"
        ]
    }
]

for rec in recommendations:
    print(f"🚀 **{rec['category']}**")
    for tip in rec['tips']:
        print(f"   • {tip}")
    print()

# Performance metrics interpretation
print("📊 **Performance Metrics Guide**")
print("   • < 1ms per check: Excellent (cached)")
print("   • 1-5ms per check: Good (typical cached performance)")
print("   • 5-20ms per check: Acceptable (cold cache)")
print("   • > 20ms per check: Investigate (complex inheritance?)")
print("   • > 100ms per check: Problem (redesign needed)")

## Chapter 4: Expert Design Patterns

Let's explore advanced design patterns that solve common complex requirements.

In [ ]:
# Design Pattern 1: The "Layered Security" Pattern
cleanup_and_reset()

print("=== Design Pattern 1: Layered Security ===")
print("Use case: Multiple security layers with different access controls")
print()

# Create layered structure
# security_zones/
#   ├── public/           (everyone)
#   ├── internal/         (employees only)
#   │   └── confidential/ (terminal - managers only)
#   │       └── restricted/ (terminal - executives only)
#   └── external/         (partners)

layers = {
    "public/announcement.txt": "Public announcement",
    "internal/policy.pdf": "Internal policy",
    "internal/confidential/salary_bands.xlsx": "Salary information",
    "internal/confidential/restricted/merger_docs.pdf": "Merger documents",
    "external/partner_api.txt": "Partner API docs"
}

for rel_path, content in layers.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)

# Layer 1: Root (baseline access)
(tutorial_dir / "syft.pub.yaml").write_text("""rules:
- pattern: "public/**"
  access:
    read:
    - "*"
- pattern: "external/**"
  access:
    read:
    - partners@company.com
""")

# Layer 2: Internal (employees)
(tutorial_dir / "internal" / "syft.pub.yaml").write_text("""rules:
- pattern: "**"
  access:
    read:
    - employees@company.com
""")

# Layer 3: Confidential (terminal - managers only)
(tutorial_dir / "internal" / "confidential" / "syft.pub.yaml").write_text("""terminal: true
rules:
- pattern: "**"
  access:
    read:
    - managers@company.com
""")

# Layer 4: Restricted (terminal - executives only)
(tutorial_dir / "internal" / "confidential" / "restricted" / "syft.pub.yaml").write_text("""terminal: true
rules:
- pattern: "**"
  access:
    read:
    - executives@company.com
""")

print("Created layered security structure:")
show_yaml(tutorial_dir, "Layer 1: Baseline")
show_yaml(tutorial_dir / "internal", "Layer 2: Internal")
show_yaml(tutorial_dir / "internal" / "confidential", "Layer 3: Confidential (terminal)")
show_yaml(tutorial_dir / "internal" / "confidential" / "restricted", "Layer 4: Restricted (terminal)")

In [ ]:
# Test layered security
print("\n=== Layered Security Test ===")

security_users = [
    "public@internet.com",
    "partners@company.com", 
    "employees@company.com",
    "managers@company.com",
    "executives@company.com"
]

layer_files = {}
for rel_path in layers.keys():
    layer_files[rel_path] = tutorial_dir / rel_path

test_access_matrix(layer_files, security_users, "read")

print("\n🛡️ Security Analysis:")
print("✓ Progressive access control - each layer adds more restrictions")
print("✓ Terminal nodes prevent privilege escalation")
print("✓ Clear security boundaries between organizational levels")

In [ ]:
# Design Pattern 2: The "Content-Based Access" Pattern
cleanup_and_reset()

print("\n=== Design Pattern 2: Content-Based Access ===")
print("Use case: Different permissions based on file type and content")
print()

# Create content-based structure
content_files = {
    "documents/public_report.pdf": "Public report",
    "documents/internal_memo.pdf": "Internal memo", 
    "source_code/frontend.js": "Frontend code",
    "source_code/backend.py": "Backend code",
    "source_code/database_schema.sql": "Database schema",
    "configs/dev_config.json": "Development config",
    "configs/prod_config.json": "Production config",
    "data/user_analytics.csv": "User analytics",
    "data/financial_data.xlsx": "Financial data"
}

for rel_path, content in content_files.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)

# Root: Default access patterns
(tutorial_dir / "syft.pub.yaml").write_text("""rules:
- pattern: "**/*public*"
  access:
    read:
    - "*"  # Anything with 'public' in name
- pattern: "**/*.js"
  access:
    write:
    - frontend_devs@company.com
    read:
    - developers@company.com
- pattern: "**/*.py"
  access:
    write:
    - backend_devs@company.com
    read:
    - developers@company.com
""")

# Configs: Environment-based access
(tutorial_dir / "configs" / "syft.pub.yaml").write_text("""rules:
- pattern: "dev_*"
  access:
    write:
    - developers@company.com
- pattern: "prod_*"
  access:
    admin:
    - devops@company.com
    read:
    - senior_devs@company.com
""")

# Data: Terminal with strict access
(tutorial_dir / "data" / "syft.pub.yaml").write_text("""terminal: true
rules:
- pattern: "*analytics*"
  access:
    read:
    - data_analysts@company.com
    - product_managers@company.com
- pattern: "*financial*"
  access:
    admin:
    - finance@company.com
""")

print("Created content-based access structure:")
show_yaml(tutorial_dir, "Root: Content patterns")
show_yaml(tutorial_dir / "configs", "Configs: Environment-based")
show_yaml(tutorial_dir / "data", "Data: Terminal with strict access")

## Chapter 5: Master-Level Edge Cases

Let's explore the most complex edge cases that only experts encounter.

In [ ]:
# Edge Case 1: Circular Permission Dependencies
cleanup_and_reset()

print("=== Edge Case 1: Complex Pattern Interactions ===")
print("Challenge: Multiple overlapping patterns with different priorities")
print()

# Create a file that matches multiple complex patterns
complex_file = tutorial_dir / "projects" / "secret_project" / "src" / "test_final_version.py"
complex_file.parent.mkdir(parents=True, exist_ok=True)
complex_file.write_text("Complex file that matches many patterns")

# YAML with multiple competing patterns
(tutorial_dir / "syft.pub.yaml").write_text("""rules:
# Most specific pattern should win
- pattern: "projects/secret_project/src/test_final_version.py"
  access:
    admin:
    - specific_admin@company.com

# Second most specific
- pattern: "projects/secret_project/**/*.py"
  access:
    write:
    - secret_devs@company.com

# Less specific patterns
- pattern: "**/test_*.py"
  access:
    read:
    - qa_team@company.com

- pattern: "**/*_version.py"
  access:
    read:
    - version_managers@company.com

- pattern: "**/*.py"
  access:
    read:
    - all_developers@company.com

# Least specific
- pattern: "**"
  access:
    read:
    - employees@company.com
""")

print("Created file with multiple matching patterns:")
print(f"File: {complex_file.relative_to(tutorial_dir)}")
print("\nPotentially matching patterns:")
print("1. projects/secret_project/src/test_final_version.py (exact match)")
print("2. projects/secret_project/**/*.py (project + extension)")
print("3. **/test_*.py (test files)")
print("4. **/*_version.py (version files)")
print("5. **/*.py (all Python files)")
print("6. ** (everything)")

show_yaml(tutorial_dir, "Complex competing patterns")

In [ ]:
# Test pattern specificity resolution
syft_complex = syft_perm.open(complex_file)

print("\n=== Pattern Specificity Test ===")
print("Expected: Most specific pattern wins (exact file match)")
print()

test_users = [
    ("specific_admin@company.com", "admin", True, "Exact pattern match"),
    ("secret_devs@company.com", "write", False, "Less specific pattern"),
    ("qa_team@company.com", "read", False, "Even less specific"),
    ("all_developers@company.com", "read", False, "General pattern"),
    ("employees@company.com", "read", False, "Catch-all pattern")
]

for user, permission, expected, description in test_users:
    has_perm = getattr(syft_complex, f"has_{permission}_access")(user)
    status = "✓" if has_perm == expected else "✗ UNEXPECTED"
    
    user_short = user.split('@')[0]
    print(f"{status} {user_short} ({permission}): {has_perm} - {description}")

print("\n🎯 Pattern Specificity Rules:")
print("1. Exact file paths beat wildcards")
print("2. Fewer wildcards = higher specificity")
print("3. Deeper paths = higher specificity")
print("4. First matching rule wins within same specificity level")

In [ ]:
# Edge Case 2: Terminal Node Inheritance Chains
cleanup_and_reset()

print("\n=== Edge Case 2: Terminal Node Chains ===")
print("Challenge: Multiple terminal nodes in inheritance path")
print()

# Create nested terminal structure
# root/ (admin access)
#   └── dept/ (terminal - dept access)
#       └── team/ (terminal - team access)
#           └── project/ (terminal - project access)
#               └── file.txt

nested_file = tutorial_dir / "dept" / "team" / "project" / "file.txt"
nested_file.parent.mkdir(parents=True, exist_ok=True)
nested_file.write_text("Deeply nested file")

# Root: Admin access
(tutorial_dir / "syft.pub.yaml").write_text("""rules:
- pattern: "**"
  access:
    admin:
    - root_admin@company.com
""")

# First terminal: Department level
(tutorial_dir / "dept" / "syft.pub.yaml").write_text("""terminal: true
rules:
- pattern: "**"
  access:
    write:
    - dept_manager@company.com
""")

# Second terminal: Team level
(tutorial_dir / "dept" / "team" / "syft.pub.yaml").write_text("""terminal: true
rules:
- pattern: "**"
  access:
    write:
    - team_lead@company.com
""")

# Third terminal: Project level
(tutorial_dir / "dept" / "team" / "project" / "syft.pub.yaml").write_text("""terminal: true
rules:
- pattern: "*.txt"
  access:
    read:
    - project_member@company.com
""")

print("Created nested terminal structure:")
print("root/ → dept/ (terminal) → team/ (terminal) → project/ (terminal) → file.txt")

# Test which terminal wins
syft_nested = syft_perm.open(nested_file)

terminal_users = [
    ("root_admin@company.com", "admin", False, "Blocked by first terminal"),
    ("dept_manager@company.com", "write", False, "Blocked by closer terminal"),
    ("team_lead@company.com", "write", False, "Blocked by closest terminal"),
    ("project_member@company.com", "read", True, "Closest terminal wins")
]

print("\n=== Terminal Chain Test ===")
for user, permission, expected, description in terminal_users:
    has_perm = getattr(syft_nested, f"has_{permission}_access")(user)
    status = "✓" if has_perm == expected else "✗ UNEXPECTED"
    
    user_short = user.split('@')[0]
    print(f"{status} {user_short} ({permission}): {has_perm} - {description}")

print("\n🔒 Terminal Chain Rules:")
print("1. Closest terminal to file wins")
print("2. Terminals completely block parent permissions")
print("3. Multiple terminals create nested isolation")
print("4. No permission accumulation across terminals")

## Chapter 6: Comprehensive Best Practices

The culmination of everything you've learned - expert-level best practices.

In [ ]:
print("🎓 SYFTBOX PERMISSIONS MASTERY GUIDE")
print("=" * 50)
print()

best_practices = {
    "🏗️ Architecture Design": [
        "Design permission hierarchy before implementing",
        "Use terminal nodes for clear security boundaries",
        "Keep directory depth reasonable (≤ 6 levels)",
        "Group related files for permission efficiency",
        "Plan for scalability from the start"
    ],
    
    "🎯 Pattern Strategy": [
        "Start with specific patterns, add general ones as fallbacks", 
        "Use exact paths for critical files",
        "Prefer descriptive patterns over catch-all '**'",
        "Order rules by specificity in YAML files",
        "Test pattern matches with real file paths"
    ],
    
    "🔒 Security Guidelines": [
        "Apply principle of least privilege",
        "Use terminal nodes to prevent privilege escalation", 
        "Implement defense in depth with multiple layers",
        "Regular audit permissions with automated tools",
        "Log and monitor permission changes"
    ],
    
    "⚡ Performance Optimization": [
        "Place YAML files close to target files",
        "Use caching effectively (don't clear unnecessarily)",
        "Minimize complex pattern matching",
        "Batch permission checks when possible",
        "Profile permission performance in production"
    ],
    
    "🐛 Debugging Strategies": [
        "Use _check_permission_with_reasons() for detailed analysis",
        "Build comprehensive test suites for edge cases",
        "Create permission audit tools for large systems",
        "Document complex permission logic",
        "Test inheritance paths thoroughly"
    ],
    
    "📏 File Limits Best Practices": [
        "Set reasonable size limits based on use case",
        "Use different limits for different user tiers",
        "Disable dangerous file types (symlinks in sandboxes)",
        "Combine limits with terminal nodes for isolation",
        "Monitor file size trends to adjust limits"
    ],
    
    "🔄 Maintenance & Operations": [
        "Version control all YAML configuration files",
        "Implement configuration validation",
        "Create backup and restore procedures",
        "Plan for permission migration during reorganization",
        "Document permission architecture for team"
    ]
}

for category, practices in best_practices.items():
    print(f"## {category}")
    for practice in practices:
        print(f"   ✓ {practice}")
    print()

print("🎯 **EXPERT TIP**: Always think about the user experience")
print("   • Clear error messages when access is denied")
print("   • Predictable permission behavior")
print("   • Easy permission debugging for administrators")
print("   • Documentation of permission policies")

In [ ]:
# Final Master Challenge: Design a complete system
print("\n🏆 FINAL MASTER CHALLENGE")
print("=" * 30)
print()
print("Design a permission system for a research institution with:")
print("• 3 departments (Biology, Physics, Computer Science)")
print("• 4 user types (Students, Researchers, Professors, Admins)")
print("• 3 data sensitivity levels (Public, Internal, Confidential)")
print("• File size limits based on user type")
print("• Cross-department collaboration spaces")
print("• Audit trail requirements")
print()
print("Requirements:")
print("1. Students can only access their department's public data")
print("2. Researchers can access internal data in their department")
print("3. Professors can access all data in their department")
print("4. Admins can access everything except confidential research")
print("5. Collaboration spaces allow cross-department access")
print("6. File uploads: Students (1MB), Researchers (10MB), Professors (100MB)")
print("7. Confidential data requires terminal isolation")
print()
print("This challenge combines all concepts from the tutorial series!")
print("Try implementing this system using what you've learned.")
print()
print("💡 **Hint**: Start with the permission hierarchy design,")
print("    then add terminal nodes for isolation,")
print("    and finally add file limits and collaboration spaces.")

## 🎉 Congratulations - You Are Now a SyftBox Permissions Expert!

You've completed the most comprehensive SyftBox permissions tutorial series! Here's what you've mastered:

### 🎓 **Your Expert Skills**

**✅ Part 1 - Fundamentals**
- Permission hierarchy (Read → Create → Write → Admin)
- Basic inheritance and nearest-node algorithm
- YAML structure and debugging basics

**✅ Part 2 - Patterns and Matching**
- Glob pattern mastery (`*`, `**`, `?`)
- Pattern specificity scoring and conflicts
- Advanced pattern debugging techniques

**✅ Part 3 - Inheritance and Hierarchy**
- Deep understanding of nearest-node algorithm
- Multi-level inheritance scenarios
- "No accumulation" principle mastery

**✅ Part 4 - Terminal Nodes and Limits**
- Terminal nodes for inheritance blocking
- File size and type restrictions
- Advanced security patterns

**✅ Part 5 - Complex Scenarios and Mastery**
- Multi-tenant architecture design
- Performance optimization techniques
- Expert debugging and troubleshooting
- Best practices and design patterns

### 🚀 **What You Can Do Now**

1. **Architect complex permission systems** for any organization
2. **Debug permission issues** quickly and effectively
3. **Optimize performance** for large-scale deployments
4. **Design secure multi-tenant** systems
5. **Implement advanced security patterns** with confidence

### 📚 **Your Reference Library**

Keep these notebooks as your reference:
- **Part 1**: Quick refresher on basics
- **Part 2**: Pattern matching troubleshooting
- **Part 3**: Inheritance debugging guide
- **Part 4**: Security patterns and limits
- **Part 5**: Expert techniques and best practices

### 🎯 **Next Steps**

1. **Practice** with real-world scenarios
2. **Share knowledge** with your team
3. **Contribute** to the SyftBox community
4. **Build amazing** secure applications!

---

**You've gone from zero to hero in SyftBox permissions! 🦸‍♂️🦸‍♀️**

*The power to secure any data sharing scenario is now in your hands.*

In [ ]:
# Cleanup and celebration
shutil.rmtree(tutorial_dir)
print("🎉 TUTORIAL SERIES COMPLETE! 🎉")
print()
print("You are now a SyftBox Permissions Expert!")
print("Ready to build secure, scalable permission systems.")
print()
print("Thank you for completing this comprehensive journey!")
print("Go forth and secure the digital world! 🌟")